In [ ]:
#Iterative final size calculation. Starting at 1.

    #Parameters:
    #R0 reproduction number.
    #s0 susceptibility at start season.
    #z final size (= attack rate).
    
import numpy as np

def finalSizeFunction(z,R0,s0):
    return s0*(1-np.exp(-R0*z))

def findFinalSize(R0,s0):
    
    zOld = finalSizeFunction(1,R0,s0)
    
    stepChange = 1
    while(stepChange > 0.00000001):
        zNew = finalSizeFunction(zOld,R0,s0)
        stepChange = abs(zOld-zNew)
        zOld = zNew
    
    return float(zNew)

In [ ]:
#Functions gives the susceptibility in the next season, given current susceptibility.

def giveNextSusceptibility(R0,gamma,Vc,Ve,sIn):
    #Order: epidemic, waning immunity, vaccination.
    
    finalSize = findFinalSize(R0,sIn)
    s1 = sIn - finalSize
    s2 = 1.0 - gamma*(1.0 - s1)
    s3 = (1.0-Vc*Ve)*s2
        
    return finalSize,s3

In [1]:
#Timeseries functions, deterministic.

def timeseriesDeterministic(R0, gamma, Vc, Ve, years):
        
    #Add burn-in period.
    years = range(0,len(years)+50)
    
    #Initialise output vectors.
    timeseriesFinal = [0.0]     #Fraction infected during seasonal epidemic.
    timeseriesSusc = [1.0]   #Fraction susceptibles at start of season.            

    for year in years:
        s0 = timeseriesSusc[-1]
        
        finalSize, susceptibility = giveNextSusceptibility(R0,gamma,Vc,Ve,s0)
        
        timeseriesFinal += [ finalSize ]
        timeseriesSusc += [ susceptibility ]
      
    return timeseriesFinal[51:], timeseriesSusc[51:]

In [2]:
#Timeseries functions, stochastic.

import random as rnd

def timeseriesStochastic(R0,Vc,Ve,years,average=True):
    
    #Add burn-in period.
    years = range(0,len(years)+50)
    
    #Initialise output vectors.
    timeseriesFinal = [0.0]     #Fraction infected during seasonal epidemic.
    timeseriesSusc = [1.0]   #Fraction susceptibles at start of season. 
    
    for year in years:
        
        gamma = rnd.betavariate(5,2)
        
        s0 = timeseriesSusc[-1]
        finalSize, susceptibility = giveNextSusceptibility(R0,gamma,Vc,Ve,s0)
        
        timeseriesFinal += [ finalSize ]
        timeseriesSusc += [ susceptibility ]

    
    if average:
        return np.mean(timeseriesFinal[51:])
    else:
        return timeseriesFinal[51:], timeseriesSusc[51:]

In [ ]:
#SIR-model for one season (ODEs).

import numpy as np
from scipy.integrate import odeint

def SIRseason(R0,s0):
    nu = 0.5
    beta = nu*R0
    N = 1000000.0
    stepsSeason = 1000
        
    if N*s0 < 100:
        print 'Warning: N to small'
    
    def derivative(y,t):
        #S=y[0], I=y[1], R=y[2]
        dSdt = -beta*y[0]*y[1]/N
        dIdt = beta*y[0]*y[1]/N-nu*y[1]
        dRdt = nu*y[1]
        return [dSdt, dIdt, dRdt]

    endTime = 365
    time = np.linspace(0.0,endTime,stepsSeason)
    yinit = np.array([s0*N-50.0,50.0,N-s0*N])

    y = odeint(derivative,yinit,time)
    
    proportions = np.array([[y[i,0]/N, y[i,1]/N, y[i,2]/N] for i in range(len(y))])
    return time, proportions

In [1]:
#Plot function. Removes box around plot.

def removeFrame(ax):
    ax.spines['top'].set_visible(False)
    ax.spines['right'].set_visible(False)

    for tic in ax.xaxis.get_major_ticks():
        tic.tick2On = False
    for tic in ax.yaxis.get_major_ticks():
        tic.tick2On = False

In [ ]:
#Save data to file for later import back into python.

import pickle

def writeData(filename,data):
    outfile = open(filename + '.dat', 'w' )
    pickle.dump(data, outfile)
    outfile.close()

#Load data again.
def readData(filename):
    infile = open(filename + '.dat', 'r')
    return pickle.load(infile)